In [2]:
import os
import sys
sys.path.append('../va_pipeline/')

In [3]:
import subprocess
import pandas as pd
import numpy as np

from calculate_accuracy import *
from drawbox import *
from sensor import exec_file


In [ ]:
draw_boxes(
    video_path='../samples/sparse.mp4', 
    ground_box=pd.read_csv('./test_results/config_testing/sparse_yolov5l_ground_truth.csv'),
    inference_box=pd.read_csv('./test_results/config_testing/frame_diff/sparse/diff_sparse_yolov5n_1280_720_25fps_inference.csv'),
    output_path='../samples/output_video.mp4')

In [11]:
s = 'frames: 250\n' + \
    'runtime (inference): 297.6342132091522\n' + \
    'average time per frame: 1.190536852836609'

In [16]:
def parse_mod(out_str):
    split = out_str.split('\n')
    frames = split[0].split(': ')
    runtime = split[1].split(': ')
    avg = split[2].split(': ')
    return {
        frames[0]: int(frames[1]),
        runtime[0]: float(runtime[1]),
        avg[0]: float(avg[1])
    }

In [17]:
print(parse_mod(s))

{'frames': 250, 'runtime (inference)': 297.6342132091522, 'average time per frame': 1.190536852836609}
